<a href="https://colab.research.google.com/github/davidlealo/sic_ai_2024/blob/main/007_proyecto/peliculas/%5B2%5Dmovier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hola profe no si es esto a lo que se refería pero aquí tengo algunos csv y los uní JSJSJ

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357272 sha256=d07e1c06f9e3c150fd1a0016326c6583b02cdccbd653100cbbf318edcdc76215
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import os
import urllib.request
import zipfile
from surprise import KNNBasic, Dataset, Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict
import pandas as pd

In [ ]:
df_muestra = pd.read_csv("df_muestra.csv")
df_muestra

In [ ]:
# Crear un diccionario para mapear id de película a título
movie_id_to_title = dict(zip(df_muestra['movieId'], df_muestra['title']))

In [ ]:


reader = Reader(rating_scale=(0.5, 5))  # Ajusta el rango de calificación si es diferente
data = Dataset.load_from_df(df_muestra[['userId', 'title', 'rating']], reader)

# Dividir el dataset en conjunto de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=.25)

# Crear el algoritmo KNNBasic
modelo = KNNBasic(k=40, sim_options={'name': 'cosine', 'user_based': True})

# Entrenar el algoritmo con el conjunto de entrenamiento
modelo.fit(trainset)

# Hacer predicciones en el conjunto de prueba
predictions = modelo.test(testset)

# Calcular el RMSE (Root Mean Squared Error)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

# Función para obtener las top N recomendaciones
def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Ordenar las predicciones para cada usuario y obtener las top n
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return dict(top_n)

# Obtener las top 5 recomendaciones para todos los usuarios
top_n = get_top_n(predictions, n=5)

# Mostrar las top 5 recomendaciones para el usuario 1
print("\nTop 5 recomendaciones para el usuario 1:")
for iid, rating in top_n['1']:
    print(f"Película: {movie_id_to_title[int(iid)]}, Calificación estimada: {rating:.2f}")

# Mostrar todas las predicciones para el usuario 1
print("\nTodas las predicciones para el usuario 1:")
user_1_predictions = [pred for pred in predictions if pred.uid == '1']
for prediction in sorted(user_1_predictions, key=lambda x: x.est, reverse=True)[:10]:
    print(f"Película: {movie_id_to_title[int(prediction.iid)]}, "
          f"Calificación estimada: {prediction.est:.2f}")